In [ ]:
#Calculate the realized market return and the beta series of each coin.
#The predicted target is based on the beta, so this process is necessary.
#NOTICE: The realized beta are based on the history price,and different from the target's beta.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

from tqdm import tqdm
import pickle

import gresearch_crypto

data_folder = "../input/g-research-crypto-forecasting/"

In [ ]:
crypto_df = pd.read_csv(data_folder + 'train.csv')
asset_details = pd.read_csv(data_folder + 'asset_details.csv')

In [ ]:
index_list = crypto_df['timestamp'].unique()

In [ ]:
#calculate the market r15 series

df_mkt = pd.DataFrame(index = index_list)
df_mkt[['r15','weight']] = 0
for i in tqdm(range(14)):
    df_tmp = pd.DataFrame(index = index_list)
    w = asset_details[asset_details['Asset_ID']==i].Weight.values[0]
    df_tmp['Close'] = crypto_df[crypto_df['Asset_ID'] == i].set_index('timestamp')['Close']
    df_tmp['r15'] = np.log(df_tmp['Close']/df_tmp['Close'].shift(15))*w
    df_tmp['weight'] = w
    df_mkt.loc[~df_tmp['r15'].isna(),['r15','weight']] += df_tmp[['r15','weight']].dropna()
mkt_r15=df_mkt.r15/df_mkt.weight
mkt_r15.to_pickle(f'/kaggle/working/mkt_r15.pkl')
del df_mkt

In [ ]:
#calculate the beta and save

cols = ['Count','Open','High','Low','Close','Volume','VWAP','Target']
for i in tqdm(range(14)):
    df_tmp = pd.DataFrame(index = index_list)
    df_tmp[cols] = crypto_df[crypto_df['Asset_ID'] == i].set_index('timestamp')[cols]
    df_tmp['r15'] = np.log(df_tmp['Close']/df_tmp['Close'].shift(15))
    df_tmp['mkt_r15'] = mkt_r15
    df_beta = df_tmp[['r15','mkt_r15']].dropna()
    df_tmp['beta3750'] = (df_beta.r15*df_beta.mkt_r15).rolling(3750).mean()/(df_beta.mkt_r15**2).rolling(3750).mean()
    pd.to_pickle(df_tmp, f'/kaggle/working/df_prefeatures{i}.pkl')

In [ ]:
del crypto_df

In [ ]:
df = pd.read_pickle(f'/kaggle/working/df_prefeatures1.pkl')
df